In [1]:
from astropy.io import fits
from astropy.table import Table
import matplotlib.pyplot as plt
import numpy as np
import math
import random
import pickle
import os
import time

%matplotlib inline
import matplotlib as mpl
from matplotlib import pyplot as plt
# better-looking plots
plt.rcParams['font.family'] = 'serif'
plt.rcParams['figure.figsize'] = (10.0, 8)
plt.rcParams['font.size'] = 18
mpl.ticker.AutoLocator.default_params['nbins'] = 5
mpl.ticker.AutoLocator.default_params['prune'] = 'both'

mpl.rcParams['ps.useafm'] = True
mpl.rcParams['pdf.use14corefonts'] = True
mpl.rcParams['text.usetex'] = True

from palettable.colorbrewer.sequential import Greys_9
from scipy.optimize import curve_fit

from astropy.cosmology import FlatLambdaCDM
from calc_kcor import calc_kcor
cosmo=FlatLambdaCDM(H0=70,Om0=0.3) # Use standard cosmology model from astropy.

In [2]:
os.chdir('../Debiasing/')

In [ ]:
import make_dictionaries
questions = make_dictionaries.questions
sample = Table.read('../fits/full_sample_debiased.fits')
print('Loaded galaxy data...')
a40 = Table.read('../fits/ALFALFA/a40.csv')
a40_reference = Table.read('../fits/ALFALFA/a40_ref.csv')
print('Loaded ALFALFA-40 data.')

In [ ]:
# select a sample of spiral galaxies:

p_spirals = (sample['t01_smooth_or_features_a02_features_or_disk_debiased_rh']*
             sample['t02_edgeon_a05_no_debiased_rh']*
             sample['t04_spiral_a08_spiral_debiased_rh']) > 0.5

N_spirals = (sample['t04_spiral_a08_spiral_count'] - 
             sample['t11_arms_number_a34_4_count']) >= 5

normal_depth =  ['coadd' not in s for s in sample['sample']]

spirals = sample[(N_spirals) & (p_spirals) & (normal_depth)]

In [ ]:
def get_sample_limits(z):
    
    z_max = [z]
    
    D_max=(10**6)*(np.array(cosmo.luminosity_distance(z_max))[0]) # Define the
    # maximum distance from the max redshift.
    m_limit=17 # Maximum apparent magnitude to be used to cut the sample.
    k_val = calc_kcor('r',z,'g - r',0.8)
    M_limit=m_limit - 5*(math.log10(D_max) - 1) - k_val

    return M_limit


def baldry_equation(u_r):
    if u_r < 79/38:
        return -0.95 + 0.56*(u_r)
    else:
        return -0.16 + 0.18*(u_r)
    
    
def lum_to_mag(L):
    return -2.5*math.log10(L) + 4.67


def find_nearest(array,value):
    idx = (np.abs(array-value)).argmin()
    return idx


def get_volume_and_mass_limits(z_values,full_data,z_min=0.03):
    
    mass_limit = np.logspace(9.5,12,1000)
    masses = []
    
    # First, get a set of 'reference' M + z values:
    z_references = np.linspace(0.03,0.2,1000)
    M_references = []
    for z in z_references:
        M_references.append(get_sample_limits(z))
    M_references = np.array([M_references])
    #----------------------------------------------
    # our data:
    u_r = full_data['PETROMAG_MU'] - full_data['PETROMAG_MR']
    Mr = full_data['PETROMAG_MR']
    
    for z in z_values:
        M_limit = get_sample_limits(z)
        in_volume_limit = ((full_data['REDSHIFT_1'] > z_min) &
                           (full_data['REDSHIFT_1'] <= z) &
                           (full_data['PETROMAG_MR'] <= M_limit))
        
        vl_ur = u_r[in_volume_limit]
        ur_99 = np.percentile(vl_ur,99,axis=0)
        logML = baldry_equation(ur_99)
        
        z_for_masses = []
        for mass in mass_limit:
            L_limit = (10**(-logML))*mass
            M_limit_mass = lum_to_mag(L_limit)
            i = find_nearest(M_references,M_limit_mass)
            z_limit = z_references[i]
            z_for_masses.append(z_limit)
        
        i2 = find_nearest(z,z_for_masses)
        masses.append(mass_limit[i2])
    
    log_masses = np.log10(masses)
    return log_masses

In [ ]:
z_values = np.linspace(0.031,0.05,100)
masses = get_volume_and_mass_limits(z_values,spirals)

In [ ]:
from scipy.optimize import curve_fit

def log_fun(x,m,c):
    return m*np.log10(x) + c

_ = plt.scatter(spirals['REDSHIFT_1'],spirals['LOGMSTAR_BALDRY06'],alpha=0.5,color='darkgreen',label='all galaxies')

p_mass,_ = curve_fit(log_fun,z_values,masses)
x_guide = np.linspace(0.02,0.1,100)
_ = plt.plot(x_guide,log_fun(x_guide,*p_mass),'y-',lw=3,label='$M_*$ complete')
plt.xlabel('$z$')
plt.ylabel('$M_*(99\%)$')

print('mass completeness limit: \n log(M_*)={} log(z) + {}'.format(np.round(p_mass[0],decimals=2),
                                                                   np.round(p_mass[1],decimals=2)))

plt.axis([0.025,0.1,9,11.5])
_ = plt.legend()

In [ ]:
lum_limit = get_sample_limits(0.05)
print('luminosity-limited sample = {}'.format(np.round(lum_limit,decimals=2)))
in_volume_limit = (spirals['PETROMAG_MR'] < lum_limit) & (spirals['REDSHIFT_1'] <= 0.05)

In [ ]:
# Match a40 catalogues by AGC number

def mass_from_redshift(z):
    K = 2.356e5*((3e5/70)**2) # constant for converting between flux and mass
    mass_limit = np.log10(K*(z**2)*0.72)
    return mass_limit


def find_matches(list1,list2):
    '''Find matches between 2 sets of numbers or strings'''
    match_values = set(list1).intersection(list2) # matches
    match_array = np.zeros((len(match_values),2))
    for i,m in enumerate(match_values):
        match_array[i,0] = np.where(np.in1d(list1,m))[0]
        match_array[i,1] = np.where(np.in1d(list2,m))[0]
    
    return match_array.astype(int)


z = np.linspace(0,0.06,100)
_ = plt.plot(z,mass_from_redshift(z),'y-',lw=3,label='completeness limit')
_ = plt.legend()
plt.xlabel('$z$')
plt.ylabel('$M_{HI}$')

a40_matches = find_matches(a40['AGCNr'],a40_reference['AGCNr'])
a40 = a40[a40_matches[:,0]]
a40_reference = a40_reference[a40_matches[:,1]]

_ = plt.scatter(a40_reference['z_sdss'],a40['logMsun'],color='darkgreen',alpha=0.5)

In [ ]:
# now define M_* and M_HI complete samples w.r.t z:

in_mass_limit = spirals['LOGMSTAR_BALDRY06'] >= p_mass[0]*np.log10(spirals['REDSHIFT_1']) + p_mass[1]
in_z_limit = spirals['REDSHIFT_1'] < 0.05
in_hi_limit = a40['logMsun'] >= mass_from_redshift(a40_reference['z_sdss'])

galaxies_ml = spirals[(in_mass_limit) & (in_z_limit)]
a40_ml = a40[in_hi_limit]
a40_reference_ml = a40_reference[in_hi_limit]

# match the samples:
matches = find_matches(a40_reference_ml['PhotoObjID'],galaxies_ml['dr7objid_1'])
matched_galaxies = galaxies_ml[matches[:,1]]
matched_a40 = a40_ml[matches[:,0]]
print('---> {} matches in a40'.format(len(matched_a40)))

In [ ]:
def redshift_from_gas_mass(logmass):
    K = 2.356e5*((3e5/70)**2) # constant for converting between flux and mass
    mass = 10**(logmass)
    z_limit = (mass/(0.72*K))**(0.5)
    return z_limit


def redshift_from_mass(logmass):
    mass = 10**logmass
    z_limit = (10**(np.log10(mass)-p_mass[1]))**(1/p_mass[0])
    return z_limit


def get_z_limit(log_stellar_mass,log_hi_mass,survey_limit=0.05):
    
    survey_array = np.ones(len(log_stellar_mass))*survey_limit
    z_limit_gas = redshift_from_gas_mass(log_hi_mass)
    z_limit_stellar = redshift_from_mass(log_stellar_mass)
    low_z = np.min(np.array([z_limit_gas,z_limit_stellar,survey_array]).T,axis=1)
    return low_z

In [ ]:
gas_mass = matched_a40['logMsun']
stellar_mass = matched_galaxies['LOGMSTAR_BALDRY06']
z = matched_galaxies['REDSHIFT_1']
z_limit = get_z_limit(stellar_mass,gas_mass)

In [ ]:
_ = plt.scatter(z,z_limit,color='darkgreen')
_ = plt.plot([0.03,0.05],[0.03,0.05],'y-',lw=3)
_ = plt.axis([0.03,0.05,0.03,0.05])
plt.xlabel('redshift (source)')
plt.ylabel('redshift (volume limit)')

In [ ]:
def get_densities(redshifts,source_redshifts,low_z_lim=0.03):
    
    N_full = len(source_redshifts)
    volume_full = np.max(source_redshifts)**3-low_z_lim**3
    full_range_density = (N_full/volume_full)
    density = []
    for z in redshifts:
        volume = z**3 - low_z_lim**3
        N_gal = np.sum(source_redshifts <= z)
        density.append(N_gal/volume)
    density = np.array(density)
    dens_norm = density/full_range_density
    return dens_norm

norm_factor = get_densities(z_limit,spirals[in_volume_limit]['REDSHIFT_1'])
Vmax_normalised = (z_limit**3-0.03**3)*norm_factor

In [ ]:
_ = plt.scatter(z_limit,Vmax_normalised,color='y',s=30)
plt.axis([0.03,0.05,0,1e-4])
plt.xlabel('redshift')
plt.ylabel("$V'_{\mathrm{max}}$")

In [ ]:
# look in to gas deficiency again:

def linear(x,m,c):
    return m*x + c


def get_gas_deficiency_line(log_mass,log_fraction,volumes,ax,plot=True,color='k',print_=True,fs=18):

    xy = np.array([x,y]).T
    mean_line,_ = curve_fit(linear,xy[:,0],xy[:,1],sigma=volumes,absolute_sigma=True)
    
    if plot:
        sizes = (np.min(volumes)/volumes)*100
        x_guide = np.linspace(9.5,11.5,100)
        _ = ax.scatter(xy[:,0],xy[:,1],color=color,s=sizes,alpha=0.75)
        ax.plot(x_guide,linear(x_guide,*mean_line),color=color,lw=2)
        ax.axis([9.5,11.5,-1.5,0.5])
        ax.set_xlabel('$\log(M_*)$')
        ax.set_ylabel('$\log(M_{\mathrm{HI}}/M_*)$')
        s = r'$\log(M_{{HI}}/M_*)= {}\log(M_*) + {}$'.format(np.round(mean_line[0],decimals=2),
                                                              np.round(mean_line[1],decimals=2))
        ax.text(0.95,0.95,s,transform=ax.transAxes,ha='right',va='top',fontsize=fs)
    elif print_:
        print('best fit: log(M_HI/M_*) = {}log(M_*) + {}'.format(np.round(mean_line[0],decimals=2),
                                                            np.round(mean_line[1],decimals=2)))
    
    return mean_line


x = matched_galaxies['LOGMSTAR_BALDRY06']
y = matched_a40['logMsun']-matched_galaxies['LOGMSTAR_BALDRY06']
ax = plt.subplot(111)
mean_line = get_gas_deficiency_line(x,y,Vmax_normalised,ax)

In [ ]:
# similar scatter plots for each of the galaxy samples?
colors_long = ['orange','r','m','g','b']

fig,axarr = plt.subplots(1,5,figsize=(20,4),sharex=True,sharey=True)
#axarr = axarr.ravel()
#fig.delaxes(axarr[-1])
plt.subplots_adjust(hspace=0,wspace=0)

def get_arm_assignments(data):
    answers = questions['t11_arms_number']['answers'][:-1]
    arm_columns = ['t11_arms_number_' + A + '_debiased_rh' for A in answers]
    arm_array = np.array([data[c] for c in arm_columns]).T
    arm_assignments = np.argmax(arm_array,axis=1)
    return arm_assignments

arm_assignments = get_arm_assignments(matched_galaxies)

for m in range(5):
    
    m_gals = matched_galaxies[arm_assignments == m]
    m_a40 = matched_a40[arm_assignments == m]
    m_V = Vmax_normalised[arm_assignments == m]
    x = m_gals['LOGMSTAR_BALDRY06']
    y = m_a40['logMsun']-m_gals['LOGMSTAR_BALDRY06']
    ax = axarr[m]
    mean_line = get_gas_deficiency_line(x,y,m_V,ax,plot=True,color=colors_long[m],fs=11)

In [ ]:
# How many galaxies are in the ALFALFA FOV?
import astropy.units as u
import astropy.coordinates as coord

def print_sample_size(dataset,high_z=0.05):
    
    dataset = dataset[dataset['REDSHIFT_1'] <= high_z]

    ra_min = coord.ICRS(ra=7.5,dec=10,unit=(u.hourangle, u.degree))
    ra_max = coord.ICRS(ra=16.5,dec=10,unit=(u.hourangle,u.degree))
    ra_limits = [ra_min.ra.deg,ra_max.ra.deg]
    select_ra = (dataset['ra_1'] >= ra_limits[0]) & (dataset['ra_1'] <= ra_limits[1])
    select_dec1 = (dataset['dec_1'] >= 4) & (dataset['dec_1'] <= 16)
    select_dec2 = (dataset['dec_1'] >= 24) & (dataset['dec_1'] <= 28)
    select_dec = [any(s) for s in zip(select_dec1,select_dec2)]
    select_spatial = (select_ra) & (select_dec)

    print('{}/{} of SDSS sample in the ALFALFA-40 coverage'.format(np.sum(select_spatial),len(select_spatial))
          + ' ({0:.1f}%)'.format(np.sum(select_spatial)/len(select_spatial)*100))
    
    return dataset[select_spatial]

In [ ]:
spatial_spirals = print_sample_size(spirals)
spatial_m = get_arm_assignments(spatial_spirals)

totals = []
totals_w_gas = []

for m in range(5):
    N_total = np.sum(spatial_m == m)
    N_w_gas = np.sum(arm_assignments == m)
    totals.append(N_total)
    totals_w_gas.append(N_w_gas)
    print('m={}: {}/{} ({}%)'.format(m+1,N_w_gas,N_total,np.round(N_w_gas/N_total*100,decimals=1)))
    
rects1 = plt.bar([0.75,1.75,2.75,3.75,4.75],totals,0.5,color='darkgreen')
rects2 = plt.bar([0.75,1.75,2.75,3.75,4.75],totals_w_gas,0.5,color='y')

In [ ]:
fig,axarr = plt.subplots(5,1,figsize=(10,20),sharex=True,sharey=True)
plt.subplots_adjust(hspace=0,wspace=0)

arm_assignments = get_arm_assignments(matched_galaxies)

for m in range(5):
    
    m_gals = matched_galaxies[arm_assignments == m]
    m_a40 = matched_a40[arm_assignments == m]
    m_V = Vmax_normalised[arm_assignments == m]
    
    bins = np.linspace(0,2,16)
    x = 10**(m_a40['logMsun']-m_gals['LOGMSTAR_BALDRY06'])
    x_full = 10**(matched_a40['logMsun']-matched_galaxies['LOGMSTAR_BALDRY06'])
    
    axarr[m].hist(x_full,normed=True,weights=1/Vmax_normalised,
                  color='k',alpha=0.4,histtype='stepfilled',bins=bins)
    axarr[m].hist(x,normed=True,weights=1/m_V,bins=bins,histtype='step',color=colors_long[m],lw=3)

In [ ]:
def get_gas_deficiency(log_mass,log_gas_mass,popt):
    
    f_gas = log_gas_mass - log_mass
    f_expected = popt[0]*log_mass + popt[1]
    gas_deficiency = f_expected - f_gas
    return gas_deficiency


gas_deficiency = get_gas_deficiency(matched_galaxies['LOGMSTAR_BALDRY06'],
                                    matched_a40['logMsun'],mean_line)

fig,axarr = plt.subplots(5,1,figsize=(10,20),sharex=True,sharey=True)
plt.subplots_adjust(hspace=0,wspace=0)

arm_assignments = get_arm_assignments(matched_galaxies)

for m in range(5):
    
    m_V = Vmax_normalised[arm_assignments == m]
    
    bins = np.linspace(-1,0.3,14)
    x = gas_deficiency[arm_assignments == m]
    x_full = gas_deficiency
    
    axarr[m].hist(x_full,normed=True,weights=1/Vmax_normalised,
                  color='k',alpha=0.4,histtype='stepfilled',bins=bins)
    axarr[m].hist(x,normed=True,weights=1/m_V,bins=bins,histtype='step',color=colors_long[m],lw=3)
    
plt.xlim([-1,0.3])

In [ ]:
# Use colours as proxies for the none-detections?

gas_sample = sample[(in_mass_limit) & (in_hi_limit) & (in_spiral)]
x = (gas_sample['PETROMAG_MU']-gas_sample['PETROMAG_MR'])#+gas_sample['LOGMSTAR_BALDRY06']
y = gas_sample['loghimass']-gas_sample['LOGMSTAR_BALDRY06']

_ = plt.scatter(x,y,s=15,color='k',alpha=0.75)

plt.xlim(1,3)
plt.xlabel('$u-r$')
plt.ylabel('$\log(M_{HI}/M_*)$')

plt.figure()
none_gas_sample = sample[(in_mass_limit) & (has_detection == False) & (in_spiral)]
B = np.linspace(1,3,21)
_ = plt.hist(gas_sample['PETROMAG_MU']-gas_sample['PETROMAG_MR']
             ,normed=True,color='b',alpha=0.4,histtype='stepfilled',label='has HI',bins=B)
_ = plt.hist(none_gas_sample['PETROMAG_MU']-none_gas_sample['PETROMAG_MR']
             ,normed=True,color='k',lw=2,histtype='step',label='no HI',bins=B)
_ = plt.legend()
plt.xlabel('$u-r$')